In [1]:
!pip install --no-cache-dir -e .
!pip install gmpy2

Obtaining file:///Users/han/python-shogi
  Attempting uninstall: python-shogi
    Found existing installation: python-shogi 1.0.14
    Uninstalling python-shogi-1.0.14:
      Successfully uninstalled python-shogi-1.0.14
  Running setup.py develop for python-shogi


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import shogi

from shogi.common import *
from shogi.features import *
from shogi.CSA import *
from shogi.player.policy_player import PolicyValuePlayer
from shogi.network.policy import PolicyNetwork
from shogi.player.mcts_player import MctsPlayer
from shogi import cli

import gmpy2

import random

In [3]:
!echo "#!/bin/sh\npython -m shogi.usi.usi_parallel_mcts" > parallel_mcts_player.sh
!chmod +x parallel_mcts_player.sh

In [4]:
import random

for i in range(100000):
    player1 = '/Users/han/python-shogi/parallel_mcts_player.sh'
    player2 = '/Users/han/python-shogi/parallel_mcts_player.sh'

    temp_rand1 = random.randint(10,1000)
    play_rand1 = random.randint(50,200)
    temp_rand2 = random.randint(10,1000)
    play_rand2 = random.randint(50,200)

    options1 = {
        'modelfile':'/Users/han/python-shogi/checkpoint/5_shogi_auto1_82_290362',
        'temperature': temp_rand1,
        'playout': play_rand1
    }
    options2 = {
        'modelfile':'/Users/han/python-shogi/checkpoint/5_shogi_auto1_17_60197',
        'temperature': temp_rand2,
        'playout': play_rand2
    }

    names = ['Player1','Player2']
    games = 10
    mate_win = True

    csa = './data/csa_auto'
    multi_csa = True

    draw = 125
    is_display = False

    cli.main(player1, player2, options1, options2, csa=csa, multi_csa=multi_csa, games=games, names=names, is_display=is_display)

Player1 vs Player2 start.


AttributeError: 'Board' object has no attribute 'move_from_usi'

In [9]:
cli.main(player1, player2, options1, options2, csa=csa, multi_csa=multi_csa, games=games, names=names, is_display=is_display)

Player1 vs Player2 start.
まで35手で千日手
1 of 100 games finished.
Player1 vs Player2: 0-0-1 (50.0%)
Black vs White: 0-0-1 (50.0%)
Player1 playing Black: 0-0-1 (50.0%)
Player1 playing White: 0-0-0 (0.0%)
Player2 playing Black: 0-0-0 (0.0%)
Player2 playing White: 0-0-1 (50.0%)
Player2 vs Player1 start.
まで13手で後手の反則負け
2 of 100 games finished.
Player1 vs Player2: 0-1-1 (25.0%)
Black vs White: 1-0-1 (75.0%)
Player1 playing Black: 0-0-1 (50.0%)
Player1 playing White: 0-1-0 (0.0%)
Player2 playing Black: 1-0-0 (100.0%)
Player2 playing White: 0-0-1 (50.0%)
Player1 vs Player2 start.
まで80手で千日手
3 of 100 games finished.
Player1 vs Player2: 0-1-2 (33.3%)
Black vs White: 1-0-2 (66.7%)
Player1 playing Black: 0-0-2 (50.0%)
Player1 playing White: 0-1-0 (0.0%)
Player2 playing Black: 1-0-0 (100.0%)
Player2 playing White: 0-0-2 (50.0%)
Player2 vs Player1 start.
まで60手で後手の勝ち
4 of 100 games finished.
Player1 vs Player2: 0-2-2 (25.0%)
Black vs White: 2-0-2 (75.0%)
Player1 playing Black: 0-0-2 (50.0%)
Player1 playing

KeyboardInterrupt: 

In [25]:
def xx(temp):
    temp1 = '{:0=25b}'.format(temp)
    for i, s in enumerate(temp1):
        if i%5 == 0:
            print()
        print(s, end='')
    print('')

def move_plot(n):
    t_occupied = Occupied(BB_SQUARES[n] ,  BB_SQUARES[n])

    BB_BISHOP_ATTACKS = (BB_R45_ATTACKS[n][(t_occupied.r45 >> BB_SHIFT_R45[n]) & 7] |
                        BB_L45_ATTACKS[n][(t_occupied.l45 >> BB_SHIFT_L45[n]) & 7])

    BB_ROOK_ATTACKS = (BB_RANK_ATTACKS[n][(t_occupied.bits >> (((n // 5) * 5) + 1)) & 7] |
                        BB_FILE_ATTACKS[n][(t_occupied.l90 >> (((n % 5) * 5) + 1)) & 7])

    BB_PROM_BISHOP_ATTACKS = (BB_KING_ATTACKS[n] |
                        BB_R45_ATTACKS[n][(t_occupied.r45 >> BB_SHIFT_R45[n]) & 7] |
                        BB_L45_ATTACKS[n][(t_occupied.l45 >> BB_SHIFT_L45[n]) & 7])

    BB_PROM_ROOK_ATTACKS = (BB_KING_ATTACKS[n] |
                        BB_RANK_ATTACKS[n][(t_occupied.bits >> (((n // 5) * 5) + 1)) & 7] |
                        BB_FILE_ATTACKS[n][(t_occupied.l90 >> (((n % 5) * 5) + 1)) & 7])


    print('pawn')
    xx(BB_PAWN_ATTACKS[0][n])
    print('----------------')
    print('silver')
    xx(BB_SILVER_ATTACKS[0][n])
    print('----------------')
    print('gold')
    xx(BB_GOLD_ATTACKS[0][n])
    print('----------------')

    print('bishop')
    xx(BB_BISHOP_ATTACKS)
    print('----------------')
    print('rook')
    xx(BB_ROOK_ATTACKS)
    print('----------------')
    print('p_bishop')
    xx(BB_PROM_BISHOP_ATTACKS)
    print('----------------')
    print('p_rook')
    xx(BB_PROM_ROOK_ATTACKS)
    print('----------------')



In [862]:
move_plot(1)

pawn

00000
00000
00000
00000
00000
----------------
silver

00000
00000
00000
00101
00000
----------------
gold

00000
00000
00000
00010
00101
----------------
bishop

00000
10000
01000
00101
00000
----------------
rook

00010
00010
00010
00010
11101
----------------
p_bishop

00000
10000
01000
00111
00101
----------------
p_rook

00010
00010
00010
00111
11101
----------------


In [655]:
import random
for i in range(509):
    if i % 4 == 0:
        print('')
    print('0x{0:016X},'.format(random.randint(0, 0xffffffffffffffff)), end='')
    if i % 4 == 3:
        print('')


0x0A82F6F3C159640B,0xB95B1031846C1C7F,0x69E32C60FD3643E3,0x381C66A41D4DF6CD,

0x4FCC7BACEDB68B00,0x7302AF66262C43C6,0x09A4B73AE746D4C4,0xAD1F87A976FC7D9C,

0xACA024799FD530FC,0x83FD4547EA1A90AF,0x536E97A9F7232E83,0x14832ECD2ADED7AE,

0x41971879E537BD93,0x2B9176693DA88BFA,0xD1DE7442388636D2,0xE238C62D02ADB2DA,

0x61D6B45F8614890B,0x1A628786CE61E55B,0xC842191B7EB5B343,0xAADDDFEE696CB11F,

0x3155594A8F643037,0x5853682E216EE7B3,0x08466AC278297E43,0x1A7FD0558334A9B3,

0x4EDDE2CFF1332134,0x4533A8F55193BF2D,0xC44350883750FC8C,0x0B5C95599BBDC428,

0x93D73DBC35E25B88,0x019985B723E78CE8,0x4FD4BA8B372AC19B,0xAA22EE53199B301C,

0xF092E1DE4F3E9B4C,0x140817965FF4F8A3,0x4712EE18A1662CE9,0x130FB5451390932B,

0x0127B33843967538,0xC66151F3A51C2EED,0x04B4EB791D2C9BB0,0x86775F6C56A636F0,

0x054C20DB1ABC1C61,0xEB8403E49BCD1DB9,0x2BA1D21A2BD9C5C6,0xE91DF9C71A6B7E74,

0xB8524D32BC019626,0x44A1DA2AE350E98A,0xB7EA85596E68362A,0x903FD02AC6BE2AE3,

0xE70369A537908E49,0x0EFB44161D7968C8,0xDAFD93AF780CDFD1,0x498C